##### Prediction for Term Deposit Subscription

  ---- By Shreya Bhatnagar

In [1]:
# IMPORTING LIBRARIES

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# IGNORE WARNINGS
import warnings
warnings.filterwarnings("ignore")

In [2]:
# ACCESSING TRAINING DATA

train = pd.read_csv('training_data.csv')

In [3]:
# PRINTING DATA

train

,ID,AGE,JOB,MARITAL,EDUCATION,DEFAULT,BALANCE,HOUSING,LOAN,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,TARGET
0,cl/26988,40,services,married,secondary,no,10406,no,no,cellular,21,nov,348,2,127,4,other,no
1,cl/44045,77,retired,married,tertiary,no,1047,no,no,cellular,30,jun,108,5,-1,0,NaN,no
2,cl/535,45,blue-collar,divorced,secondary,no,756,yes,no,NaN,6,may,179,2,-1,0,NaN,no
3,cl/39433,27,management,single,tertiary,no,616,yes,no,cellular,22,may,685,1,101,1,other,yes
4,cl/5241,36,management,married,tertiary,no,8564,yes,no,NaN,23,may,125,1,-1,0,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31642,cl/12363,53,services,married,primary,no,931,no,no,NaN,27,jun,159,2,-1,0,NaN,no
31643,cl/5695,50,admin.,single,secondary,no,1651,yes,no,NaN,26,may,41,2,-1,0,NaN,no
31644,cl/8006,33,technician,single,secondary,no,-244,yes,no,NaN,2,jun,63,2,-1,0,NaN,no
31645,cl/17745,52,management,divorced,tertiary,no,-299,no,no,cellular,29,jul,110,2,-1,0,NaN,no


In [4]:
# PRINTING SHAPE

train.shape

(31647, 18)

In [5]:
# PRINTING COLUMN NAMES

train.columns

Index(['ID', 'AGE', 'JOB', 'MARITAL', 'EDUCATION', 'DEFAULT', 'BALANCE',
       'HOUSING', 'LOAN', 'CONTACT', 'DAY', 'MONTH', 'DURATION', 'CAMPAIGN',
       'PDAYS', 'PREVIOUS', 'POUTCOME', 'TARGET'],
      dtype='object')

In [6]:
# EDITING TARGET VARIABLE

train.TARGET.replace(('yes', 'no'), (1, 0), inplace=True)


In [7]:
# TRAIN TEST SPLIT
train_X_variables = ['AGE', 'JOB', 'MARITAL', 'EDUCATION', 'DEFAULT', 'BALANCE',
       'HOUSING', 'DAY', 'LOAN','CONTACT', 'MONTH', 'DURATION', 'CAMPAIGN',
       'PDAYS', 'PREVIOUS', 'POUTCOME']
train_y_variables = ['TARGET']
X_train,X_test,y_train,y_test = train_test_split(train[train_X_variables], train[train_y_variables], test_size=0.33, random_state=1)

In [8]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

       AGE            JOB  MARITAL  EDUCATION DEFAULT  BALANCE HOUSING  DAY  \
31494   36         admin.  married  secondary      no        0      no   26   
1715    40    blue-collar  married  secondary      no     1313      no   14   
13701   30         admin.   single  secondary      no       72      no    3   
30868   49     management   single   tertiary      no     7443      no   19   
21217   58    blue-collar  married    primary      no     1596     yes   12   
...    ...            ...      ...        ...     ...      ...     ...  ...   
17289   29       services   single  secondary      no      487     yes    9   
5192    26    blue-collar  married    primary      no        0     yes   21   
12172   49  self-employed  married  secondary      no     -199     yes   29   
235     50    blue-collar  married    primary      no      111     yes   15   
29733   52    blue-collar  married    unknown      no     2171     yes   17   

      LOAN    CONTACT MONTH  DURATION  CAMPAIGN  PD

###### Cleaning Data

<I><u>Getting Catagorical and Numerical Columns to impute</u></I>

In [9]:
# Getting Catagorical Columns

s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

# Getting Numerical Columns

s = (X_train.dtypes != 'object')
num_cols = list(s[s].index)

#printing to check

print(object_cols)
print(num_cols)

['JOB', 'MARITAL', 'EDUCATION', 'DEFAULT', 'HOUSING', 'LOAN', 'CONTACT', 'MONTH', 'POUTCOME']
['AGE', 'BALANCE', 'DAY', 'DURATION', 'CAMPAIGN', 'PDAYS', 'PREVIOUS']


In [10]:
# Numeric Transformer

numerical_transformer = SimpleImputer(strategy='mean')

# Catagorical Transformer

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, object_cols)
    ])


In [11]:
#!pip install catboost
from catboost import CatBoostClassifier

model_cat = CatBoostClassifier(custom_loss=['Accuracy'],random_seed=42,loss_function = 'CrossEntropy')


In [12]:
# APPLYING PIPELINE

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model_cat)
                             ])
my_pipeline.fit(X_train, y_train)
preds = my_pipeline.predict(X_test)
preds_training = my_pipeline.predict(X_train)

print(preds)
print(sum(preds))
print(len(preds) - sum(preds))
print((sum(preds))/(len(preds) - sum(preds)))

0:	learn: 0.6589964	total: 214ms	remaining: 3m 33s
1:	learn: 0.6277992	total: 246ms	remaining: 2m 2s
2:	learn: 0.5946908	total: 274ms	remaining: 1m 31s
3:	learn: 0.5693062	total: 302ms	remaining: 1m 15s
4:	learn: 0.5436121	total: 337ms	remaining: 1m 7s
5:	learn: 0.5211868	total: 369ms	remaining: 1m 1s
6:	learn: 0.5002355	total: 401ms	remaining: 56.9s
7:	learn: 0.4832766	total: 425ms	remaining: 52.7s
8:	learn: 0.4629565	total: 453ms	remaining: 49.8s
9:	learn: 0.4462167	total: 477ms	remaining: 47.3s
10:	learn: 0.4323106	total: 505ms	remaining: 45.4s
11:	learn: 0.4174953	total: 533ms	remaining: 43.9s
12:	learn: 0.4054981	total: 560ms	remaining: 42.5s
13:	learn: 0.3949535	total: 585ms	remaining: 41.2s
14:	learn: 0.3834873	total: 611ms	remaining: 40.1s
15:	learn: 0.3748613	total: 636ms	remaining: 39.1s
16:	learn: 0.3678651	total: 662ms	remaining: 38.3s
17:	learn: 0.3605342	total: 686ms	remaining: 37.4s
18:	learn: 0.3522851	total: 707ms	remaining: 36.5s
19:	learn: 0.3443550	total: 730ms	rema

169:	learn: 0.2105790	total: 4.15s	remaining: 20.3s
170:	learn: 0.2104577	total: 4.17s	remaining: 20.2s
171:	learn: 0.2103221	total: 4.19s	remaining: 20.2s
172:	learn: 0.2101794	total: 4.22s	remaining: 20.2s
173:	learn: 0.2099889	total: 4.24s	remaining: 20.1s
174:	learn: 0.2096149	total: 4.26s	remaining: 20.1s
175:	learn: 0.2094806	total: 4.28s	remaining: 20.1s
176:	learn: 0.2093143	total: 4.31s	remaining: 20s
177:	learn: 0.2091063	total: 4.33s	remaining: 20s
178:	learn: 0.2089055	total: 4.35s	remaining: 19.9s
179:	learn: 0.2086187	total: 4.37s	remaining: 19.9s
180:	learn: 0.2084429	total: 4.39s	remaining: 19.9s
181:	learn: 0.2083205	total: 4.42s	remaining: 19.8s
182:	learn: 0.2082089	total: 4.44s	remaining: 19.8s
183:	learn: 0.2080125	total: 4.46s	remaining: 19.8s
184:	learn: 0.2078958	total: 4.48s	remaining: 19.8s
185:	learn: 0.2077525	total: 4.5s	remaining: 19.7s
186:	learn: 0.2074231	total: 4.53s	remaining: 19.7s
187:	learn: 0.2072552	total: 4.55s	remaining: 19.6s
188:	learn: 0.207

334:	learn: 0.1905588	total: 8.5s	remaining: 16.9s
335:	learn: 0.1904733	total: 8.52s	remaining: 16.8s
336:	learn: 0.1903348	total: 8.55s	remaining: 16.8s
337:	learn: 0.1902274	total: 8.57s	remaining: 16.8s
338:	learn: 0.1901535	total: 8.59s	remaining: 16.8s
339:	learn: 0.1900669	total: 8.62s	remaining: 16.7s
340:	learn: 0.1899057	total: 8.65s	remaining: 16.7s
341:	learn: 0.1898331	total: 8.68s	remaining: 16.7s
342:	learn: 0.1897086	total: 8.71s	remaining: 16.7s
343:	learn: 0.1896398	total: 8.74s	remaining: 16.7s
344:	learn: 0.1895677	total: 8.77s	remaining: 16.6s
345:	learn: 0.1894544	total: 8.79s	remaining: 16.6s
346:	learn: 0.1892977	total: 8.81s	remaining: 16.6s
347:	learn: 0.1892095	total: 8.83s	remaining: 16.5s
348:	learn: 0.1891099	total: 8.86s	remaining: 16.5s
349:	learn: 0.1890433	total: 8.88s	remaining: 16.5s
350:	learn: 0.1889854	total: 8.9s	remaining: 16.5s
351:	learn: 0.1889219	total: 8.92s	remaining: 16.4s
352:	learn: 0.1888239	total: 8.94s	remaining: 16.4s
353:	learn: 0.

494:	learn: 0.1767547	total: 12.3s	remaining: 12.5s
495:	learn: 0.1766475	total: 12.3s	remaining: 12.5s
496:	learn: 0.1765764	total: 12.3s	remaining: 12.5s
497:	learn: 0.1765017	total: 12.3s	remaining: 12.4s
498:	learn: 0.1764261	total: 12.4s	remaining: 12.4s
499:	learn: 0.1763498	total: 12.4s	remaining: 12.4s
500:	learn: 0.1762522	total: 12.4s	remaining: 12.4s
501:	learn: 0.1762019	total: 12.4s	remaining: 12.3s
502:	learn: 0.1761467	total: 12.5s	remaining: 12.3s
503:	learn: 0.1760810	total: 12.5s	remaining: 12.3s
504:	learn: 0.1759381	total: 12.5s	remaining: 12.3s
505:	learn: 0.1758751	total: 12.5s	remaining: 12.2s
506:	learn: 0.1758102	total: 12.5s	remaining: 12.2s
507:	learn: 0.1757029	total: 12.6s	remaining: 12.2s
508:	learn: 0.1756168	total: 12.6s	remaining: 12.1s
509:	learn: 0.1755539	total: 12.6s	remaining: 12.1s
510:	learn: 0.1755150	total: 12.6s	remaining: 12.1s
511:	learn: 0.1754644	total: 12.7s	remaining: 12.1s
512:	learn: 0.1753973	total: 12.7s	remaining: 12s
513:	learn: 0.

659:	learn: 0.1663125	total: 16.1s	remaining: 8.29s
660:	learn: 0.1662540	total: 16.1s	remaining: 8.27s
661:	learn: 0.1661764	total: 16.1s	remaining: 8.24s
662:	learn: 0.1661300	total: 16.2s	remaining: 8.22s
663:	learn: 0.1660466	total: 16.2s	remaining: 8.2s
664:	learn: 0.1659940	total: 16.2s	remaining: 8.17s
665:	learn: 0.1659400	total: 16.3s	remaining: 8.15s
666:	learn: 0.1658639	total: 16.3s	remaining: 8.13s
667:	learn: 0.1658005	total: 16.3s	remaining: 8.1s
668:	learn: 0.1657272	total: 16.3s	remaining: 8.08s
669:	learn: 0.1656893	total: 16.4s	remaining: 8.06s
670:	learn: 0.1656383	total: 16.4s	remaining: 8.04s
671:	learn: 0.1655826	total: 16.4s	remaining: 8.01s
672:	learn: 0.1655097	total: 16.4s	remaining: 7.99s
673:	learn: 0.1654812	total: 16.5s	remaining: 7.96s
674:	learn: 0.1654270	total: 16.5s	remaining: 7.93s
675:	learn: 0.1653838	total: 16.5s	remaining: 7.91s
676:	learn: 0.1653151	total: 16.5s	remaining: 7.88s
677:	learn: 0.1652895	total: 16.5s	remaining: 7.86s
678:	learn: 0.

825:	learn: 0.1580994	total: 19.9s	remaining: 4.18s
826:	learn: 0.1580649	total: 19.9s	remaining: 4.16s
827:	learn: 0.1580388	total: 19.9s	remaining: 4.13s
828:	learn: 0.1579977	total: 19.9s	remaining: 4.11s
829:	learn: 0.1579512	total: 19.9s	remaining: 4.08s
830:	learn: 0.1579244	total: 20s	remaining: 4.06s
831:	learn: 0.1578122	total: 20s	remaining: 4.04s
832:	learn: 0.1577706	total: 20s	remaining: 4.01s
833:	learn: 0.1577195	total: 20s	remaining: 3.99s
834:	learn: 0.1576818	total: 20.1s	remaining: 3.96s
835:	learn: 0.1576413	total: 20.1s	remaining: 3.94s
836:	learn: 0.1575954	total: 20.1s	remaining: 3.92s
837:	learn: 0.1575481	total: 20.1s	remaining: 3.89s
838:	learn: 0.1575115	total: 20.1s	remaining: 3.87s
839:	learn: 0.1574653	total: 20.2s	remaining: 3.84s
840:	learn: 0.1574327	total: 20.2s	remaining: 3.82s
841:	learn: 0.1573885	total: 20.2s	remaining: 3.79s
842:	learn: 0.1573488	total: 20.2s	remaining: 3.77s
843:	learn: 0.1573091	total: 20.3s	remaining: 3.74s
844:	learn: 0.157283

986:	learn: 0.1513495	total: 23.4s	remaining: 309ms
987:	learn: 0.1512745	total: 23.5s	remaining: 285ms
988:	learn: 0.1512243	total: 23.5s	remaining: 261ms
989:	learn: 0.1511506	total: 23.5s	remaining: 238ms
990:	learn: 0.1510744	total: 23.5s	remaining: 214ms
991:	learn: 0.1510093	total: 23.6s	remaining: 190ms
992:	learn: 0.1509546	total: 23.6s	remaining: 166ms
993:	learn: 0.1508976	total: 23.6s	remaining: 143ms
994:	learn: 0.1508523	total: 23.6s	remaining: 119ms
995:	learn: 0.1508160	total: 23.7s	remaining: 95.1ms
996:	learn: 0.1508060	total: 23.7s	remaining: 71.3ms
997:	learn: 0.1507809	total: 23.7s	remaining: 47.5ms
998:	learn: 0.1507601	total: 23.8s	remaining: 23.8ms
999:	learn: 0.1507159	total: 23.8s	remaining: 0us
[0 0 0 ... 1 0 0]
874
9570
0.09132706374085685


In [13]:
print(accuracy_score(y_test,preds))
print(accuracy_score(y_train,preds_training))

0.906644963615473
0.9438287034853559


In [14]:
# IMPORTING PREDICTION DATA

test = pd.read_csv('test_data.csv')

In [15]:
test

,ID,AGE,JOB,MARITAL,EDUCATION,DEFAULT,BALANCE,HOUSING,LOAN,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME
0,cl/37199,32,blue-collar,married,secondary,no,1779,yes,no,cellular,13,may,191,1,342,2,failure
1,cl/20751,45,management,married,tertiary,no,324,no,no,cellular,13,aug,258,4,-1,0,NaN
2,cl/554,33,blue-collar,married,secondary,no,-10,yes,no,NaN,6,may,123,1,-1,0,NaN
3,cl/37464,32,technician,single,secondary,no,271,yes,no,cellular,13,may,80,2,268,2,other
4,cl/35798,34,technician,divorced,secondary,no,136,yes,no,cellular,8,may,136,2,-1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13559,cl/9059,55,retired,divorced,tertiary,no,1218,no,yes,NaN,5,jun,145,1,-1,0,NaN
13560,cl/20804,36,management,single,secondary,no,0,no,no,cellular,13,aug,333,5,-1,0,NaN
13561,cl/19560,30,technician,single,secondary,no,-127,no,no,cellular,7,aug,76,3,-1,0,NaN
13562,cl/34206,33,management,single,tertiary,no,417,yes,no,cellular,4,may,170,2,87,1,failure


In [16]:
final_train_X = train[train_X_variables]
final_train_y = train[train_y_variables]
final_test_X = test[train_X_variables]

In [17]:
# Getting Catagorical Columns

s = (final_train_X.dtypes == 'object')
object_cols = list(s[s].index)

# Getting Numerical Columns

s = (final_train_X.dtypes != 'object')
num_cols = list(s[s].index)

#printing to check

print(object_cols)
print(num_cols)

['JOB', 'MARITAL', 'EDUCATION', 'DEFAULT', 'HOUSING', 'LOAN', 'CONTACT', 'MONTH', 'POUTCOME']
['AGE', 'BALANCE', 'DAY', 'DURATION', 'CAMPAIGN', 'PDAYS', 'PREVIOUS']


In [18]:
# Numeric Transformer

numerical_transformer = SimpleImputer(strategy='mean')

# Catagorical Transformer

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, object_cols)
    ])


In [19]:
# APPLYING PIPELINE

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model_cat)
                             ])
my_pipeline.fit(final_train_X, final_train_y)
preds = my_pipeline.predict(final_test_X)



0:	learn: 0.6586324	total: 42.4ms	remaining: 42.3s
1:	learn: 0.6294269	total: 63.9ms	remaining: 31.9s
2:	learn: 0.6027703	total: 90.3ms	remaining: 30s
3:	learn: 0.5722784	total: 116ms	remaining: 28.9s
4:	learn: 0.5470036	total: 142ms	remaining: 28.4s
5:	learn: 0.5247971	total: 170ms	remaining: 28.2s
6:	learn: 0.5025395	total: 196ms	remaining: 27.8s
7:	learn: 0.4807159	total: 225ms	remaining: 27.9s
8:	learn: 0.4607203	total: 253ms	remaining: 27.8s
9:	learn: 0.4438091	total: 281ms	remaining: 27.8s
10:	learn: 0.4282837	total: 307ms	remaining: 27.6s
11:	learn: 0.4144461	total: 337ms	remaining: 27.8s
12:	learn: 0.4034409	total: 370ms	remaining: 28.1s
13:	learn: 0.3910505	total: 418ms	remaining: 29.5s
14:	learn: 0.3806963	total: 476ms	remaining: 31.3s
15:	learn: 0.3714042	total: 524ms	remaining: 32.2s
16:	learn: 0.3639406	total: 557ms	remaining: 32.2s
17:	learn: 0.3561368	total: 582ms	remaining: 31.8s
18:	learn: 0.3464509	total: 608ms	remaining: 31.4s
19:	learn: 0.3391198	total: 640ms	remain

162:	learn: 0.2123499	total: 4.43s	remaining: 22.8s
163:	learn: 0.2122425	total: 4.46s	remaining: 22.7s
164:	learn: 0.2121084	total: 4.49s	remaining: 22.7s
165:	learn: 0.2119620	total: 4.52s	remaining: 22.7s
166:	learn: 0.2118277	total: 4.56s	remaining: 22.7s
167:	learn: 0.2117326	total: 4.59s	remaining: 22.7s
168:	learn: 0.2116020	total: 4.62s	remaining: 22.7s
169:	learn: 0.2115077	total: 4.65s	remaining: 22.7s
170:	learn: 0.2113304	total: 4.68s	remaining: 22.7s
171:	learn: 0.2112037	total: 4.71s	remaining: 22.7s
172:	learn: 0.2109942	total: 4.74s	remaining: 22.7s
173:	learn: 0.2108487	total: 4.77s	remaining: 22.6s
174:	learn: 0.2106472	total: 4.79s	remaining: 22.6s
175:	learn: 0.2105626	total: 4.82s	remaining: 22.6s
176:	learn: 0.2104203	total: 4.84s	remaining: 22.5s
177:	learn: 0.2102289	total: 4.88s	remaining: 22.5s
178:	learn: 0.2100979	total: 4.9s	remaining: 22.5s
179:	learn: 0.2098992	total: 4.93s	remaining: 22.4s
180:	learn: 0.2097567	total: 4.95s	remaining: 22.4s
181:	learn: 0

325:	learn: 0.1961190	total: 9.09s	remaining: 18.8s
326:	learn: 0.1960527	total: 9.12s	remaining: 18.8s
327:	learn: 0.1959460	total: 9.15s	remaining: 18.7s
328:	learn: 0.1958813	total: 9.18s	remaining: 18.7s
329:	learn: 0.1957946	total: 9.21s	remaining: 18.7s
330:	learn: 0.1957084	total: 9.24s	remaining: 18.7s
331:	learn: 0.1956599	total: 9.26s	remaining: 18.6s
332:	learn: 0.1956125	total: 9.29s	remaining: 18.6s
333:	learn: 0.1954943	total: 9.32s	remaining: 18.6s
334:	learn: 0.1953443	total: 9.35s	remaining: 18.6s
335:	learn: 0.1952363	total: 9.37s	remaining: 18.5s
336:	learn: 0.1951933	total: 9.4s	remaining: 18.5s
337:	learn: 0.1951070	total: 9.43s	remaining: 18.5s
338:	learn: 0.1950196	total: 9.45s	remaining: 18.4s
339:	learn: 0.1949575	total: 9.48s	remaining: 18.4s
340:	learn: 0.1948679	total: 9.51s	remaining: 18.4s
341:	learn: 0.1947982	total: 9.54s	remaining: 18.3s
342:	learn: 0.1946961	total: 9.56s	remaining: 18.3s
343:	learn: 0.1945965	total: 9.59s	remaining: 18.3s
344:	learn: 0

490:	learn: 0.1853847	total: 13.8s	remaining: 14.3s
491:	learn: 0.1853269	total: 13.8s	remaining: 14.3s
492:	learn: 0.1852614	total: 13.9s	remaining: 14.3s
493:	learn: 0.1851995	total: 13.9s	remaining: 14.2s
494:	learn: 0.1851381	total: 13.9s	remaining: 14.2s
495:	learn: 0.1850829	total: 13.9s	remaining: 14.2s
496:	learn: 0.1849823	total: 14s	remaining: 14.1s
497:	learn: 0.1849253	total: 14s	remaining: 14.1s
498:	learn: 0.1849164	total: 14s	remaining: 14.1s
499:	learn: 0.1848755	total: 14s	remaining: 14s
500:	learn: 0.1848062	total: 14.1s	remaining: 14s
501:	learn: 0.1847489	total: 14.1s	remaining: 14s
502:	learn: 0.1847100	total: 14.1s	remaining: 13.9s
503:	learn: 0.1846249	total: 14.1s	remaining: 13.9s
504:	learn: 0.1845482	total: 14.2s	remaining: 13.9s
505:	learn: 0.1844781	total: 14.2s	remaining: 13.9s
506:	learn: 0.1844201	total: 14.2s	remaining: 13.8s
507:	learn: 0.1843592	total: 14.2s	remaining: 13.8s
508:	learn: 0.1843119	total: 14.3s	remaining: 13.8s
509:	learn: 0.1842583	tota

652:	learn: 0.1765970	total: 18.2s	remaining: 9.68s
653:	learn: 0.1765539	total: 18.2s	remaining: 9.65s
654:	learn: 0.1765193	total: 18.3s	remaining: 9.62s
655:	learn: 0.1764831	total: 18.3s	remaining: 9.59s
656:	learn: 0.1764414	total: 18.3s	remaining: 9.56s
657:	learn: 0.1764131	total: 18.3s	remaining: 9.53s
658:	learn: 0.1763364	total: 18.4s	remaining: 9.5s
659:	learn: 0.1762962	total: 18.4s	remaining: 9.47s
660:	learn: 0.1762695	total: 18.4s	remaining: 9.44s
661:	learn: 0.1762168	total: 18.4s	remaining: 9.41s
662:	learn: 0.1761658	total: 18.5s	remaining: 9.38s
663:	learn: 0.1761213	total: 18.5s	remaining: 9.36s
664:	learn: 0.1760790	total: 18.5s	remaining: 9.33s
665:	learn: 0.1760076	total: 18.5s	remaining: 9.3s
666:	learn: 0.1759686	total: 18.6s	remaining: 9.27s
667:	learn: 0.1759264	total: 18.6s	remaining: 9.24s
668:	learn: 0.1758839	total: 18.6s	remaining: 9.21s
669:	learn: 0.1758435	total: 18.6s	remaining: 9.18s
670:	learn: 0.1757976	total: 18.7s	remaining: 9.15s
671:	learn: 0.

816:	learn: 0.1694838	total: 22.7s	remaining: 5.08s
817:	learn: 0.1694428	total: 22.7s	remaining: 5.05s
818:	learn: 0.1694026	total: 22.7s	remaining: 5.02s
819:	learn: 0.1693390	total: 22.7s	remaining: 4.99s
820:	learn: 0.1692782	total: 22.8s	remaining: 4.96s
821:	learn: 0.1692295	total: 22.8s	remaining: 4.93s
822:	learn: 0.1691832	total: 22.8s	remaining: 4.91s
823:	learn: 0.1691224	total: 22.8s	remaining: 4.88s
824:	learn: 0.1690882	total: 22.9s	remaining: 4.85s
825:	learn: 0.1690592	total: 22.9s	remaining: 4.82s
826:	learn: 0.1690213	total: 22.9s	remaining: 4.79s
827:	learn: 0.1689950	total: 22.9s	remaining: 4.77s
828:	learn: 0.1689892	total: 23s	remaining: 4.74s
829:	learn: 0.1689543	total: 23s	remaining: 4.71s
830:	learn: 0.1689230	total: 23s	remaining: 4.68s
831:	learn: 0.1688737	total: 23.1s	remaining: 4.65s
832:	learn: 0.1688391	total: 23.1s	remaining: 4.63s
833:	learn: 0.1687748	total: 23.1s	remaining: 4.6s
834:	learn: 0.1687387	total: 23.1s	remaining: 4.57s
835:	learn: 0.16869

981:	learn: 0.1630921	total: 27.1s	remaining: 498ms
982:	learn: 0.1630274	total: 27.2s	remaining: 470ms
983:	learn: 0.1629814	total: 27.2s	remaining: 442ms
984:	learn: 0.1629415	total: 27.2s	remaining: 415ms
985:	learn: 0.1629110	total: 27.3s	remaining: 387ms
986:	learn: 0.1629083	total: 27.3s	remaining: 359ms
987:	learn: 0.1628728	total: 27.3s	remaining: 332ms
988:	learn: 0.1628399	total: 27.3s	remaining: 304ms
989:	learn: 0.1627886	total: 27.4s	remaining: 276ms
990:	learn: 0.1627560	total: 27.4s	remaining: 249ms
991:	learn: 0.1627314	total: 27.4s	remaining: 221ms
992:	learn: 0.1627004	total: 27.4s	remaining: 193ms
993:	learn: 0.1626751	total: 27.5s	remaining: 166ms
994:	learn: 0.1626174	total: 27.5s	remaining: 138ms
995:	learn: 0.1625619	total: 27.5s	remaining: 110ms
996:	learn: 0.1625239	total: 27.5s	remaining: 82.9ms
997:	learn: 0.1624863	total: 27.6s	remaining: 55.2ms
998:	learn: 0.1624482	total: 27.6s	remaining: 27.6ms
999:	learn: 0.1624218	total: 27.6s	remaining: 0us
